In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [3]:
option = webdriver.ChromeOptions()
option.add_argument("--incognito")
driverPath = "E:\drivers\chromedriver"

In [13]:
browser = webdriver.Chrome(executable_path=driverPath, options=option)
browser.get("https://twitter.com/login")


#login to twitter
try:
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.NAME,"session[username_or_email]"))    
    )
except TimeoutException:
    element=False

if(element):
    print('yess')
    browser.find_element_by_name("session[username_or_email]").send_keys("***")
    browser.find_element_by_name("session[password]").send_keys("****")
    browser.find_element_by_class_name("css-1dbjc4n").click()
    
print(browser.title)



yess
Login on Twitter / Twitter


In [14]:
#search for a name
name = input("Enter the name to be searched")
li = name.split()
if(len(li)>1):
    name = "%20".join(li)
query = "https://twitter.com/search?q="+name+"&src=typed_query&f=user"
print(query)
browser.get(query)

Enter the name to be searchedVipasha
https://twitter.com/search?q=Vipasha&src=typed_query&f=user


In [15]:
#lenOfPage = browser.execute_script("""window.scrollTo(0,document.body.scrollHeight);
#                                    var lenOfPage=document.body.scrollHeight;
#                                    return lenOfPage;""")

accNo = int(input("Specify the no of scrolls\n(one scroll on an avrage will get you 8-10 records) :"))
i=0
idList = []
while(i<(accNo)):
    source_data = browser.page_source
    soup = bs(source_data,'lxml')
    res = soup.find_all('div', class_='css-1dbjc4n r-18u37iz r-1wbh5a2')
    if i%2==0 or i==0:
        for q in res:
            idList.append(q.text)
        for w in range(3):
            idList.pop()
    time.sleep(3)
    print(i)
    lenOfPage = browser.execute_script("""window.scrollTo(0,document.body.scrollHeight+100);
                            var lenOfPage=document.body.scrollHeight;
                            return lenOfPage""")
    print(lenOfPage)
    i+=1


Specify the no of scrolls
(one scroll on an avrage will get you 8-10 records) :6
0
2793
1
4681
2
4731
3
4731
4
4731
5
4731


In [16]:
idList = list(dict.fromkeys(idList))

In [17]:
for i in idList:
    print(i)

@BipashaFanClub
@Bipsfan4ever
@NuraLoveBipasha
@BipashaPatgiri
@thelabellife
@Bipashaa
@tweetChirrup
@Utterly_Vips
@VipashaMalhotra
@thoughtstorm1
@BipashaCNN
@bimo20
@VipashaP
@vipashabansal
@LaraBipashaFan
@BipashaMukherj7


In [18]:
len(idList)

16

In [19]:
def to_number(text):
    factor = 1
    if text[-1]=='K':
        return int(float(text[:-1])*1000)
    elif text[-1]=='M':
        return int(float(text[:-1])*1000000)
    elif ',' in text:
        return int("".join(text.split(',')))
    else:
        return int(text)

In [25]:
data = []

for iidd in range(len(idList)):
    req = "https://twitter.com/"+idList[iidd][1:]
    print(idList[iidd][1:])
    browser.get(req)
    time.sleep(4)
    s1 = browser.page_source
    sp1 = bs(s1,'lxml')
    re1 = sp1.find('div', class_="css-1dbjc4n r-1cad53l r-779j7e r-1b9bua6")
    #css-1dbjc4n r-1cad53l r-779j7e r-1b9bua6
    #details
    detail_dic = {}
    #id
    detail_dic['TwitterId'] = idList[iidd][1:]

    #name
    for i in re1.find("div", class_="css-1dbjc4n r-1awozwy r-18u37iz r-dnmrzs"):
        name = i.text
        detail_dic['Name'] = name
        if i.find('g'):
            verified = "Verified"
        else:
            verified = "Not Verified"
        detail_dic['Verified'] = verified

    #bio
    bio_loc_join = re1.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")
    bio = bio_loc_join.text
    detail_dic['Bio'] = bio

    #Number of tweets
    tweets = sp1.find('div', class_="css-901oao css-bfa6kz r-111h2gw r-1qd0xha r-1enofrn r-16dba41 r-1o9f8vr r-bcqeeo r-qvutc0")
    tw = tweets.text.split()[0]
    detail_dic['#_of_tweets'] = to_number(tw)

    #followers and following
    follow =  re1.find('div', class_="css-1dbjc4n r-18u37iz").find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")
    following = follow[0].text
    follower = follow[2].text
    detail_dic['Following'] = to_number(following)
    detail_dic['Follower'] = to_number(follower)
    
    data.append(detail_dic)

BipashaFanClub
Bipsfan4ever
NuraLoveBipasha
BipashaPatgiri
thelabellife
Bipashaa
tweetChirrup
Utterly_Vips
VipashaMalhotra
thoughtstorm1
BipashaCNN
bimo20
VipashaP
vipashabansal
LaraBipashaFan
BipashaMukherj7


In [26]:
columns = ['TwitterId', 'Name', 'Verified','Bio', '#_of_tweets','Following','Follower']
df = pd.DataFrame(data, columns = columns)

In [27]:
df

,TwitterId,Name,Verified,Bio,#_of_tweets,Following,Follower
0,BipashaFanClub,Bipasha Basu FanClub,Not Verified,Follow,11600,1,20200
1,Bipsfan4ever,Bipasha Basu FC,Not Verified,Follow,6420,181,11100
2,NuraLoveBipasha,Nurana Haciyeva,Not Verified,Follow,22700,207,5238
3,BipashaPatgiri,Bipasha Patgiri,Not Verified,Follow,136,564,77
4,thelabellife,The Label Life,Verified,Follow,3228,109,19400
5,Bipashaa,Bipasha,Not Verified,Follow,866,534,249
6,tweetChirrup,tweetChirrup$Bipasha,Not Verified,Follow,175,99,67
7,Utterly_Vips,Vipasha Kashyap,Not Verified,Follow,3567,298,244
8,VipashaMalhotra,Vipasha Malhotra,Not Verified,Follow,326,84,59
9,thoughtstorm1,Vipasha Tilak,Not Verified,Follow,44,63,44
